#### Import libraries

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import warnings
import sys
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader

sys.path.append("../")
from pipeline import data
from pipeline.config import CONF
from pipeline.data import plots
from pipeline.data import io
from pipeline.data import inspection
from pipeline.data import preprocess

# To suppress all warnings
warnings.filterwarnings("ignore")

# black is a code formatter (see https://github.com/psf/black).
# It will automatically format the code you write in the cells imposing consistent Python style.
%load_ext jupyter_black
# matplotlib style file
# Template for style file: https://matplotlib.org/stable/tutorials/introductory/customizing.html#customizing-with-style-sheets
plt.style.use("../matplotlib_style.txt")
pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.expand_frame_repr", False)  # Prevent wrapping

### Load raw data

This takes about 1 minute for the first time.

In [ ]:
if CONF.data.process_raw_data and not CONF.data.loaded_raw_data:
    # Load raw data
    (
        Installed_Capacity_Germany_Raw,
        Prices_Europe_Raw,
        Realised_Supply_Germany_Raw,
        Realised_Demand_Germany_Raw,
        Weather_Data_Germany_Raw,
        Weather_Data_Germany_2022_Raw,
    ) = data.load_data(CONF=CONF)
    CONF.data.loaded_raw_data = True

In [ ]:
if CONF.data.process_raw_data:
    Installed_Capacity_Germany = Installed_Capacity_Germany_Raw.copy()
    Prices_Europe = Prices_Europe_Raw.copy()
    Realised_Supply_Germany = Realised_Supply_Germany_Raw.copy()
    Realised_Demand_Germany = Realised_Demand_Germany_Raw.copy()
    Weather_Data_Germany = Weather_Data_Germany_Raw.copy()
    Weather_Data_Germany_2022 = Weather_Data_Germany_2022_Raw.copy()

### Inspect raw data

##### Inspect missingness

In [ ]:
if CONF.data.process_raw_data:
    # Generate profile reports
    if CONF.data.inspect:
        data.save_data_inspection(
            Installed_Capacity_Germany=Installed_Capacity_Germany,
            Prices_Europe=Prices_Europe,
            Realised_Supply_Germany=Realised_Supply_Germany,
            Realised_Demand_Germany=Realised_Demand_Germany,
            Weather_Data_Germany=Weather_Data_Germany,
            Weather_Data_Germany_2022=Weather_Data_Germany_2022,
            CONF=CONF,
            data_type="raw",
        )

##### Inspect resolution

In [ ]:
if CONF.data.process_raw_data:
    inspection.date_range_and_resolution_dfs(
        Installed_Capacity_Germany=Installed_Capacity_Germany,
        Prices_Europe=Prices_Europe,
        Realised_Supply_Germany=Realised_Supply_Germany,
        Realised_Demand_Germany=Realised_Demand_Germany,
        Weather_Data_Germany=Weather_Data_Germany,
    )

#### Plot raw data

In [ ]:
if CONF.data.plot:
    plots.plot_df(
        Installed_Capacity_Germany,
        "Installed_Capacity_Germany",
        CONF,
        processed_data=False,
    )
    plots.plot_df(Prices_Europe, "Prices_Europe", CONF, processed_data=False)
    plots.plot_df(
        Realised_Supply_Germany, "Realised_Supply_Germany", CONF, processed_data=False
    )
    plots.plot_df(
        Realised_Demand_Germany, "Realised_Demand_Germany", CONF, processed_data=False
    )
    plots.plot_df(
        Weather_Data_Germany,
        "Weather_Data_Germany",
        CONF,
        date_col=io.DATE_COLUMNS_WEATHER[-1],
        drop_date_cols=io.DATE_COLUMNS_WEATHER,
        processed_data=False,
    )

### Raw data pipeline

##### Merging weather data frames

In [ ]:
if CONF.data.process_raw_data:
    # Remove the data for 2022 from the original dataframe
    Weather_Data_Germany = Weather_Data_Germany[
        Weather_Data_Germany["time"].dt.year != 2022
    ]

    # Concatenate the filtered original dataframe with the 2022 data
    Weather_Data_Germany = pd.concat(
        [Weather_Data_Germany, Weather_Data_Germany_2022], ignore_index=True
    )

##### Fill NaN

In [ ]:
if CONF.data.process_raw_data:
    Processed_Installed_Capacity_Germany = data.process_na_values(
        Installed_Capacity_Germany, CONF
    )
    Processed_Prices_Europe = data.process_na_values(Prices_Europe, CONF)
    Processed_Realised_Supply_Germany = data.process_na_values(
        Realised_Supply_Germany, CONF
    )
    Processed_Realised_Demand_Germany = data.process_na_values(
        Realised_Demand_Germany, CONF
    )
    Processed_Weather_Data_Germany = data.process_na_values(Weather_Data_Germany, CONF)

#### Aggregate weather data

In [ ]:
if CONF.data.process_raw_data:
    Processed_Weather_Data_Germany = preprocess.aggregate_weather_data(
        Processed_Weather_Data_Germany, ["forecast_origin", "time"]
    )

#### Decrease demand and supply's time resolution

In [ ]:
if CONF.data.process_raw_data:
    Processed_Realised_Demand_Germany = Processed_Realised_Demand_Germany[
        Processed_Realised_Demand_Germany["Date to"].dt.minute == 0
    ]
    Processed_Realised_Demand_Germany["Date from"] = Processed_Realised_Demand_Germany[
        "Date to"
    ] - pd.Timedelta(hours=1)
    Processed_Realised_Supply_Germany = Processed_Realised_Supply_Germany[
        Processed_Realised_Supply_Germany["Date to"].dt.minute == 0
    ]
    Processed_Realised_Supply_Germany["Date from"] = Processed_Realised_Supply_Germany[
        "Date to"
    ] - pd.Timedelta(hours=1)

#### Increase time resolution of capacity

In [ ]:
new_row = {
    "Date from": pd.Timestamp("2022-12-31 23:00:00"),
    "Date to": pd.Timestamp("2023-01-01 00:00:00"),
}
new_row_df = pd.DataFrame([new_row])
Processed_Installed_Capacity_Germany_hourly = pd.concat(
    [Processed_Installed_Capacity_Germany, new_row_df], ignore_index=True
)

Processed_Installed_Capacity_Germany_hourly = (
    Processed_Installed_Capacity_Germany_hourly.set_index("Date from")
)
Processed_Installed_Capacity_Germany_hourly = (
    Processed_Installed_Capacity_Germany_hourly.resample("H").mean()
)
Processed_Installed_Capacity_Germany_hourly.reset_index(inplace=True)
Processed_Installed_Capacity_Germany_hourly["Date to"] = (
    Processed_Installed_Capacity_Germany_hourly["Date from"] + pd.Timedelta(hours=1)
)
Processed_Installed_Capacity_Germany = Processed_Installed_Capacity_Germany_hourly
Processed_Installed_Capacity_Germany = Processed_Installed_Capacity_Germany.fillna(
    method="ffill"
)
inspection.date_range_and_resolution(
    Processed_Installed_Capacity_Germany, io.DATE_COLUMNS
)

#### Trim rows of every df to have same range

In [ ]:
# trim first row of Processed_Weather_Data_Germany
Processed_Weather_Data_Germany = Processed_Weather_Data_Germany[
    Processed_Weather_Data_Germany["time"]
    != Processed_Weather_Data_Germany["time"].min()
]

# trim last row of every other df
Processed_Installed_Capacity_Germany = Processed_Installed_Capacity_Germany[
    Processed_Installed_Capacity_Germany["Date to"]
    != Processed_Installed_Capacity_Germany["Date to"].max()
]
Processed_Prices_Europe = Processed_Prices_Europe[
    Processed_Prices_Europe["Date to"] != Processed_Prices_Europe["Date to"].max()
]
Processed_Realised_Supply_Germany = Processed_Realised_Supply_Germany[
    Processed_Realised_Supply_Germany["Date to"]
    != Processed_Realised_Supply_Germany["Date to"].max()
]
Processed_Realised_Demand_Germany = Processed_Realised_Demand_Germany[
    Processed_Realised_Demand_Germany["Date to"]
    != Processed_Realised_Demand_Germany["Date to"].max()
]

#### Patch time saving

In [ ]:
Processed_Prices_Europe = preprocess.patch_time_saving(Processed_Prices_Europe)
Processed_Realised_Demand_Germany = preprocess.patch_time_saving(
    Processed_Realised_Demand_Germany
)
Processed_Realised_Supply_Germany = preprocess.patch_time_saving(
    Processed_Realised_Supply_Germany
)

#### Normalize data

In [ ]:
if CONF.data.process_raw_data and CONF.data.normalize_data:
    print("Split data in train, val and test")
    Processed_Installed_Capacity_Germany = preprocess.split_data(
        df=Processed_Installed_Capacity_Germany, column_name=io.DATE_COLUMNS[-1]
    )
    Processed_Prices_Europe = preprocess.split_data(
        df=Processed_Prices_Europe, column_name=io.DATE_COLUMNS[-1]
    )
    Processed_Realised_Supply_Germany = preprocess.split_data(
        df=Processed_Realised_Supply_Germany, column_name=io.DATE_COLUMNS[-1]
    )
    Processed_Realised_Demand_Germany = preprocess.split_data(
        df=Processed_Realised_Demand_Germany, column_name=io.DATE_COLUMNS[-1]
    )
    Processed_Weather_Data_Germany = preprocess.split_data(
        df=Processed_Weather_Data_Germany, column_name=io.DATE_COLUMNS_WEATHER[0]
    )

    print("Normalizing data")
    (
        Processed_Installed_Capacity_Germany,
        Processed_Installed_Capacity_Germany_Scalers,
    ) = preprocess.normalize_data(
        df=Processed_Installed_Capacity_Germany,
        ignore_features=io.DATE_COLUMNS,
        constant=CONF.data.price_normalization_constant,
    )

    Processed_Prices_Europe, Processed_Prices_Europe_Scalers = (
        preprocess.normalize_data(
            df=Processed_Prices_Europe,
            ignore_features=io.DATE_COLUMNS,
            constant=CONF.data.price_normalization_constant,
        )
    )
    Processed_Realised_Supply_Germany, Processed_Realised_Supply_Germany_Scalers = (
        preprocess.normalize_data(
            df=Processed_Realised_Supply_Germany, ignore_features=io.DATE_COLUMNS
        )
    )
    Processed_Realised_Demand_Germany, Processed_Realised_Demand_Germany_Scalers = (
        preprocess.normalize_data(
            df=Processed_Realised_Demand_Germany, ignore_features=io.DATE_COLUMNS
        )
    )
    Processed_Weather_Data_Germany, Processed_Weather_Data_Germany_Scalers = (
        preprocess.normalize_data(
            df=Processed_Weather_Data_Germany,
            ignore_features=io.DATE_COLUMNS_WEATHER + ["longitude", "latitude"],
        )
    )

    print("Remove train, test, val columns, again")
    Processed_Installed_Capacity_Germany = Processed_Installed_Capacity_Germany.drop(
        ["train", "val", "test"], axis=1
    )
    Processed_Prices_Europe = Processed_Prices_Europe.drop(
        ["train", "val", "test"], axis=1
    )
    Processed_Realised_Supply_Germany = Processed_Realised_Supply_Germany.drop(
        ["train", "val", "test"], axis=1
    )
    Processed_Realised_Demand_Germany = Processed_Realised_Demand_Germany.drop(
        ["train", "val", "test"], axis=1
    )
    Processed_Weather_Data_Germany = Processed_Weather_Data_Germany.drop(
        ["train", "val", "test"], axis=1
    )

## Inspect processed data

### Inspect missingness

In [ ]:
# Generate profile reports

if CONF.data.inspect:
    data.save_data_inspection(
        Installed_Capacity_Germany=Processed_Installed_Capacity_Germany,
        Prices_Europe=Processed_Prices_Europe,
        Realised_Supply_Germany=Processed_Realised_Supply_Germany,
        Realised_Demand_Germany=Processed_Realised_Demand_Germany,
        Weather_Data_Germany=Processed_Weather_Data_Germany,
        CONF=CONF,
        data_type="preprocessed",
    )

### Plot processed data

In [ ]:
if CONF.data.plot:
    plots.plot_df(
        Processed_Installed_Capacity_Germany, "Installed_Capacity_Germany", CONF
    )
    plots.plot_df(Processed_Prices_Europe, "Prices_Europe", CONF)
    plots.plot_df(Processed_Realised_Supply_Germany, "Realised_Supply_Germany", CONF)
    plots.plot_df(Processed_Realised_Demand_Germany, "Realised_Demand_Germany", CONF)
    plots.plot_df(
        Processed_Weather_Data_Germany,
        "Weather_Data_Germany",
        CONF,
        date_col=io.DATE_COLUMNS_WEATHER[-1],
        drop_date_cols=io.DATE_COLUMNS_WEATHER,
    )

### Inspect time's resolution

In [ ]:
inspection.date_range_and_resolution_dfs(
    Installed_Capacity_Germany=Processed_Installed_Capacity_Germany,
    Prices_Europe=Processed_Prices_Europe,
    Realised_Supply_Germany=Processed_Realised_Supply_Germany,
    Realised_Demand_Germany=Processed_Realised_Demand_Germany,
    Weather_Data_Germany=Processed_Weather_Data_Germany,
    processed=True,
)

### Data Unit Tests

In [ ]:
dfs = [
    Processed_Installed_Capacity_Germany,
    Processed_Prices_Europe,
    Processed_Realised_Supply_Germany,
    Processed_Realised_Demand_Germany,
]

raw_dfs = [
    Installed_Capacity_Germany_Raw,
    Prices_Europe_Raw,
    Realised_Supply_Germany_Raw,
    Realised_Demand_Germany_Raw,
]


# assert all data frames have same length.
def test_dataframe_lengths(dfs):
    expected_length = len(dfs[0])
    for df in dfs[1:]:
        assert len(df) == expected_length, "DataFrames have different lengths"


test_dataframe_lengths(dfs)


# assert all data frames have have same time resolution
def test_dataframe_resolutions(dfs):
    date_column = "Date from"
    expected_resolution = dfs[0][date_column].diff().dropna().mode()[0]
    for df in dfs[1:]:
        current_resolution = df[date_column].diff().dropna().mode()[0]
        assert (
            current_resolution == expected_resolution
        ), "DataFrames have different time resolutions"
    current_resolution = (
        Processed_Weather_Data_Germany["time"].diff().dropna().mode()[0]
    )
    assert (
        current_resolution == expected_resolution
    ), "DataFrames have different time resolutions"


test_dataframe_resolutions(dfs)


# Assert every row of every df has the same "Date to"
def test_date_to_consistency(dfs):
    date_to_values = dfs[0]["Date to"].values
    for i, df in enumerate(dfs[1:]):
        assert all(
            df["Date to"].values == date_to_values
        ), f"Mismatch in 'Date to' values across DataFrames {i + 1}"
    assert np.all(
        Processed_Weather_Data_Germany["time"].values == date_to_values
    ), "DataFrames have different time resolutions"


test_date_to_consistency(dfs)


# assert that not cell is missed
def test_no_missing_cells(dfs):
    for i, df in enumerate(dfs + [Processed_Weather_Data_Germany]):
        for column in df.columns:
            assert (
                df[column].isnull().sum() == 0
            ), f"Missing values found in column '{column}' of DataFrame at index {i}"


test_no_missing_cells(dfs)


# assert that raw dfs and normal dfs have same set of columns
def test_same_columns(raw_dfs, processed_dfs, ignore_columns=None):
    if ignore_columns is None:
        ignore_columns = set()

    for raw_df, processed_df in zip(raw_dfs, processed_dfs):
        raw_columns = set(raw_df.columns) - ignore_columns
        processed_columns = set(processed_df.columns) - ignore_columns

        assert (
            raw_columns == processed_columns
        ), f"Column mismatch between raw and processed DataFrames. Missing {raw_columns - processed_columns} or {processed_columns - raw_columns}"


# Example usage of the function, ignoring 'train', 'val', and 'test'
test_same_columns(
    raw_dfs,
    dfs,
    ignore_columns={
        "∅ Neighbouring DE/LU [€/MWh]",
        "Hungary [€/MWh]",
        "Poland [€/MWh]",
        "DE/AT/LU [€/MWh]",
    },
)

## Data loader for supply forecasting 

##### Merge data together to a single frame

In [ ]:
def add_prefix(df, prefix):
    """
    Add a prefix to all column names except the merge key.
    """
    return df.rename(
        columns={
            col: prefix + col if col not in io.DATE_COLUMNS else col
            for col in df.columns
        }
    )


Suffix_Processed_Prices_Europe = add_prefix(Processed_Prices_Europe, "prices_")
Suffix_Processed_Installed_Capacity_Germany = add_prefix(
    Processed_Installed_Capacity_Germany, "capacity_"
)
Suffix_Processed_Realised_Supply_Germany = add_prefix(
    Processed_Realised_Supply_Germany, "supply_"
)
Suffix_Processed_Realised_Demand_Germany = add_prefix(
    Processed_Realised_Demand_Germany, "demand_"
)
Suffix_Processed_Weather_Data_Germany = add_prefix(
    Processed_Weather_Data_Germany, "weather_"
)
Suffix_Processed_Weather_Data_Germany = Suffix_Processed_Weather_Data_Germany.rename(
    columns={"weather_time": "Date to"}
)

# Now perform the merge
df = pd.merge(
    Suffix_Processed_Prices_Europe,
    Suffix_Processed_Installed_Capacity_Germany,
    on=io.DATE_COLUMNS,
    how="inner",
)
df = pd.merge(
    df, Suffix_Processed_Realised_Supply_Germany, on=io.DATE_COLUMNS, how="inner"
)
df = pd.merge(
    df, Suffix_Processed_Realised_Demand_Germany, on=io.DATE_COLUMNS, how="inner"
)
df = pd.merge(
    df, Suffix_Processed_Weather_Data_Germany, on=io.DATE_COLUMNS[-1], how="inner"
)
df.head()

In [ ]:
if CONF.data.plot:
    plots.plot_df(df, "Final Dataframe", CONF, figsize=(150, 30))

##### Split train, val, test

In [ ]:
df = preprocess.split_data(df=df, column_name=io.DATE_COLUMNS[-1])

##### Torch's datasets

In [ ]:
import torch
from torch.utils.data import Dataset
from pipeline.config import CONF


class TimeSeriesDataset(Dataset):
    def __init__(self, df: pd.DataFrame, CONF):
        self.dataframe = df
        self.features = CONF.model.features
        self.targets = CONF.model.targets

        # Set lag based on the horizon
        if CONF.model.horizon == 1:
            self.lag = CONF.model.supply1.lag
        elif CONF.model.horizon == 24:
            self.lag = CONF.model.supply24.lag
        else:
            raise ValueError("Horizon must be 1 or 24 hours")

        self.horizon = CONF.model.horizon
        # Calculate total samples considering the number of lag and the forecasting horizon
        self.total_samples = len(df) - (self.lag + self.horizon)

    def __len__(self):
        return self.total_samples

    def __getitem__(self, idx):
        idx += self.lag  # Adjust start index to accommodate initial lag
        # Collect inputs using lag
        inputs = []
        for lag in range(self.lag):
            lag_idx = idx - lag
            inputs.append(
                self.dataframe.loc[lag_idx, self.features].values.astype(float)
            )  # n_features
        inputs = torch.tensor(inputs)  # Shape: [lag, n_features]

        # Collect targets based on the horizon
        targets_idx = idx + self.horizon
        targets = torch.tensor(
            self.dataframe.loc[targets_idx, self.targets].values.astype(float)
        )

        return inputs, targets

#### Torch's dataloaders

In [ ]:
train_df = df[df["train"]].drop(CONF.model.ignore_columns, axis=1)
val_df = df[df["val"]].drop(CONF.model.ignore_columns, axis=1)
test_df = df[df["test"]].drop(CONF.model.ignore_columns, axis=1)

train_dataset = TimeSeriesDataset(train_df, CONF=CONF)
val_dataset = TimeSeriesDataset(val_df, CONF=CONF)
test_dataset = TimeSeriesDataset(test_df, CONF=CONF)


train_loader = DataLoader(train_dataset, batch_size=CONF.model.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CONF.model.batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=CONF.model.batch_size, shuffle=False)

In [ ]:
for inputs, targets in train_loader:
    print(inputs.shape, targets.shape)
    break